# GPT-2 finetuning on heavy metal lyrics

This notebook provides the code for finetuning OpenAI's GPT-2 model on heavy metal lyrics.
I used simple Python package called [`gpt-2-simple`](https://github.com/minimaxir/gpt-2-simple) that wraps existing model fine-tuning and generation scripts for OpenAI's GPT-2 text generation model (specifically the "small" 124M and "medium" 355M hyperparameter versions)

Hardware used for thi task:

- g4dn.xlarge instance on AWS (NVIDIA Tesla P4 GPU 8 GB, 4 vCPU, 16 GB RAM)

## Import libraries

In [ ]:
import gpt_2_simple as gpt2
import pandas as pd
import tensorflow as tf
import os
import shutil

from tqdm import tqdm
tqdm.pandas()

## Import and preprocess dataset

Set all the lyrics in a unique file and delimit each text with the strings `<|startoftext|>` and `<|endoftext|>`.

In [2]:
metal_lyrics_df = pd.read_csv("../datasets/metal_dataset_clean_eng.csv", sep="|")

print(f"Number of songs: {len(metal_lyrics_df)}")
metal_lyrics_df.sample(10)

Number of songs: 195202


,artist,album,album_type,release_year,title,track_no,lyrics,lang_iso,lang_name
184382,Venom,Fallen Angels,Studio,2011,Death Be Thy Name,11,When you’re least expecting my call\nOr at the...,eng,English
180549,U.D.O.,Rev-Raptor,Studio,2011,Renegade,3,Contract Double Dealing\nHired Killer Calls\nS...,eng,English
161171,Slapshot,Back On The Map,Studio,1984,Might Makes Right,9,You can take your ideas and throw them out the...,eng,English
33235,Capsize,I've Been Tearing Myself Apart,EP,2014,Face First,1,"I failed myself\nAgain today\nLiar, liar\nIt's...",eng,English
92351,Impaled Nazarene,Rapture,Studio,1998,Vitutation,11,"Agonized, beaten up\nThis feeling ain't just g...",eng,English
43951,The Carrier,Blind To What Is Right,Studio,2010,In Silence Together,6,I'm watching everyone around me slowly kill th...,eng,English
138852,Phobia,Get Up And Kill!,EP,2004,Revolutionary's Hell,7,I've felt the cold cell\nA distinctive place t...,eng,English
89255,Howling Syn,Forebearers Of Dusk,Studio,2002,Witches Of Avalon,9,"The witches are sometimes crazy, sometimes gen...",eng,English
153889,Scarve,Luminiferous,Studio,2002,Infertile Ways,9,We have reached a point in existence\nWhere th...,eng,English
149365,Rudra,The Aryan Crusade,Studio,2001,I,13,"Knowledge frees you, Action binds you\nSelf Ig...",eng,English


In [3]:
lyrics_corpus = "../datasets/lyrics_corpus.txt"
with open(lyrics_corpus, 'w') as out_file:
    for index, row in tqdm(metal_lyrics_df.iterrows(), total=metal_lyrics_df.shape[0]):
        text = str(row["lyrics"]).replace("\\n", "\n")
        out_file.write("<|startoftext|>" + text + "<|endoftext|>" + "\n")

100%|██████████| 195202/195202 [00:18<00:00, 10462.77it/s]


## Set constants

Note that the model chosen is the "medium" 355M!

In [ ]:
MODEL_DIR = '../models'
MODEL_NAME = '355M'
CHECKPOINT_DIR = '../models/deep-metal/checkpoints'
FINETUNED_MODEL_NAME = 'deep-metal'
SAMPLE_DIR = '../models/deep-metal/samples'
RUN_NAME = 'run1'

## Download model and encode dataset

In [3]:
gpt2.download_gpt2(model_dir=MODEL_DIR, model_name=MODEL_NAME)

Fetching checkpoint: 1.05Mit [00:00, 253Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 14.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 338Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:49, 28.6Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 353Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 19.4Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 25.3Mit/s]                                                      


In [6]:
gpt2.encode_dataset(
    "../datasets/lyrics_corpus.txt", 
    model_dir=MODEL_DIR,
    model_name=MODEL_NAME, 
    out_path="../datasets/lyrics_corpus_encoded.npz"
)

  0%|          | 0/1 [00:00<?, ?it/s]

Reading files


100%|██████████| 1/1 [03:29<00:00, 209.07s/it]


Writing ../datasets/lyrics_corpus_encoded.npz


## Finetune the model

Train it for 5000 steps.

In [7]:
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(
    sess,
    dataset='../datasets/lyrics_corpus_encoded.npz',
    model_dir=MODEL_DIR,
    model_name=MODEL_NAME,
    checkpoint_dir=CHECKPOINT_DIR,
    steps=5000,
    batch_size=2,
    learning_rate=0.0001,
    restore_from='fresh',
    run_name=RUN_NAME,
    print_every=10,
    sample_every=200,
    sample_num=1,
    save_every=500,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint ../models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from ../models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.33s/it]


dataset has 48625368 tokens
Training...
[10 | 37.83] loss=3.19 avg=3.19
[20 | 67.53] loss=2.83 avg=3.01
[30 | 97.82] loss=2.50 avg=2.84
[40 | 128.64] loss=3.65 avg=3.05
[50 | 159.74] loss=3.08 avg=3.05
[60 | 191.35] loss=3.36 avg=3.11
[70 | 223.20] loss=2.49 avg=3.02
[80 | 254.99] loss=2.89 avg=3.00
[90 | 286.79] loss=3.06 avg=3.01
[100 | 318.67] loss=3.28 avg=3.03
[110 | 350.49] loss=3.24 avg=3.05
[120 | 382.31] loss=3.64 avg=3.11
[130 | 414.19] loss=2.83 avg=3.08
[140 | 446.03] loss=2.14 avg=3.01
[150 | 477.89] loss=2.83 avg=3.00
[160 | 509.71] loss=2.93 avg=2.99
[170 | 541.52] loss=3.36 avg=3.02
[180 | 573.33] loss=3.08 avg=3.02
[190 | 605.20] loss=3.01 avg=3.02
[200 | 637.07] loss=2.57 avg=3.00
======== SAMPLE 1 ========

My life
Telling you
It's
Never
I'll
Go
Wherever you go
Never get lost
Never
Get
Lost
I hope
I can say
I was there too
I had you
Don't
Don't
Carry
All the way to
Eternity
My life
Telling you
It's
Never
I'll
Get
Lost
I hope
I can say
I was there too
I had you
Don't


...

[3210 | 10632.43] loss=2.84 avg=2.86
[3220 | 10664.32] loss=3.38 avg=2.87
[3230 | 10696.16] loss=2.29 avg=2.86
[3240 | 10728.02] loss=3.03 avg=2.86
[3250 | 10759.92] loss=2.65 avg=2.86
[3260 | 10791.76] loss=2.57 avg=2.86
[3270 | 10823.60] loss=3.15 avg=2.86
[3280 | 10855.49] loss=2.87 avg=2.86
[3290 | 10887.36] loss=3.17 avg=2.86
[3300 | 10919.22] loss=1.82 avg=2.85
[3310 | 10951.08] loss=2.99 avg=2.85
[3320 | 10983.01] loss=2.62 avg=2.85
[3330 | 11014.89] loss=3.12 avg=2.85
[3340 | 11046.78] loss=2.59 avg=2.85
[3350 | 11078.62] loss=2.68 avg=2.85
[3360 | 11110.51] loss=2.65 avg=2.85
[3370 | 11142.42] loss=2.69 avg=2.85
[3380 | 11174.34] loss=2.50 avg=2.84
[3390 | 11206.21] loss=3.03 avg=2.84
[3400 | 11238.12] loss=2.64 avg=2.84
======== SAMPLE 1 ========
endoftext|>
<|startoftext|>When I'm tired of all the things in life the worst part
Is that the worst part is knowing you
There's nothing in my life and I can't stand
But when I close my eyes you'll never leave me
And the worst part i

[3610 | 11958.40] loss=3.45 avg=2.84
[3620 | 11990.24] loss=3.13 avg=2.84
[3630 | 12022.10] loss=1.78 avg=2.83
[3640 | 12053.96] loss=2.79 avg=2.83
[3650 | 12085.81] loss=3.59 avg=2.84
[3660 | 12117.75] loss=2.71 avg=2.84
[3670 | 12149.59] loss=2.73 avg=2.84
[3680 | 12181.42] loss=3.00 avg=2.84
[3690 | 12213.31] loss=2.94 avg=2.84
[3700 | 12245.15] loss=2.89 avg=2.84
[3710 | 12277.02] loss=2.78 avg=2.84
[3720 | 12308.88] loss=3.03 avg=2.84
[3730 | 12340.76] loss=3.39 avg=2.85
[3740 | 12372.63] loss=2.09 avg=2.84
[3750 | 12404.51] loss=2.45 avg=2.84
[3760 | 12436.37] loss=2.28 avg=2.83
[3770 | 12468.25] loss=2.86 avg=2.83
[3780 | 12500.14] loss=2.42 avg=2.83
[3790 | 12531.99] loss=2.75 avg=2.83
[3800 | 12563.80] loss=2.53 avg=2.82
======== SAMPLE 1 ========


The blood from his hands is his only blood, all blood must have a blood...
You must have some blood...

The blood will rain...

From blood... from blood

And a sacrifice comes to me
I'll take my time and I will get some more
And I'

[4010 | 13284.38] loss=2.98 avg=2.79
[4020 | 13316.30] loss=2.92 avg=2.79
[4030 | 13348.18] loss=2.93 avg=2.79
[4040 | 13380.06] loss=3.00 avg=2.79
[4050 | 13411.96] loss=2.72 avg=2.79
[4060 | 13443.83] loss=3.72 avg=2.80
[4070 | 13475.72] loss=3.25 avg=2.80
[4080 | 13507.60] loss=2.31 avg=2.80
[4090 | 13539.47] loss=3.16 avg=2.80
[4100 | 13571.35] loss=2.66 avg=2.80
[4110 | 13603.21] loss=2.89 avg=2.80
[4120 | 13635.11] loss=2.62 avg=2.80
[4130 | 13666.97] loss=2.88 avg=2.80
[4140 | 13698.86] loss=2.03 avg=2.79
[4150 | 13730.72] loss=2.68 avg=2.79
[4160 | 13762.59] loss=3.37 avg=2.80
[4170 | 13794.45] loss=2.57 avg=2.80
[4180 | 13826.33] loss=2.16 avg=2.79
[4190 | 13858.20] loss=3.62 avg=2.80
[4200 | 13890.10] loss=1.78 avg=2.79
======== SAMPLE 1 ========
ented of the sun

Dawn

Gathered around my bed
Facing my own shadow
My mother will be missed
But not, not for long
Filled with an inner pain
My head is spinning round
But I will keep my dream
I will keep my dream

My mother was never

[4410 | 14607.29] loss=2.00 avg=2.79
[4420 | 14639.17] loss=2.38 avg=2.79
[4430 | 14671.08] loss=3.31 avg=2.79
[4440 | 14702.95] loss=2.80 avg=2.79
[4450 | 14734.84] loss=2.57 avg=2.79
[4460 | 14766.73] loss=2.26 avg=2.79
[4470 | 14798.55] loss=2.75 avg=2.78
[4480 | 14830.47] loss=3.23 avg=2.79
[4490 | 14862.37] loss=3.00 avg=2.79
[4500 | 14894.18] loss=2.88 avg=2.79
Saving ../models/deep-metal/checkpoints/run1/model-4500
[4510 | 14928.92] loss=3.64 avg=2.80
[4520 | 14960.77] loss=3.02 avg=2.80
[4530 | 14992.60] loss=3.47 avg=2.81
[4540 | 15024.47] loss=3.17 avg=2.81
[4550 | 15056.33] loss=2.56 avg=2.81
[4560 | 15088.17] loss=2.73 avg=2.81
[4570 | 15120.08] loss=3.06 avg=2.81
[4580 | 15151.94] loss=2.12 avg=2.81
[4590 | 15183.80] loss=2.33 avg=2.80
[4600 | 15215.63] loss=2.50 avg=2.80
======== SAMPLE 1 ========
 tyrant
Is the last of the chosen

It is your choice and your choice alone
I would not change it<|endoftext|>
<|startoftext|>For everything that you did I am forever torn
The da

[4810 | 15932.62] loss=2.78 avg=2.81
[4820 | 15964.51] loss=2.66 avg=2.81
[4830 | 15996.39] loss=2.86 avg=2.81
[4840 | 16028.23] loss=2.80 avg=2.81
[4850 | 16060.10] loss=2.53 avg=2.81
[4860 | 16091.99] loss=2.53 avg=2.80
[4870 | 16123.86] loss=2.02 avg=2.79
[4880 | 16155.70] loss=3.01 avg=2.80
[4890 | 16187.53] loss=3.19 avg=2.80
[4900 | 16219.44] loss=2.57 avg=2.80
[4910 | 16251.31] loss=2.98 avg=2.80
[4920 | 16283.15] loss=3.17 avg=2.80
[4930 | 16314.98] loss=2.43 avg=2.80
[4940 | 16346.86] loss=2.65 avg=2.80
[4950 | 16378.73] loss=2.50 avg=2.80
[4960 | 16410.58] loss=3.60 avg=2.80
[4970 | 16442.44] loss=3.38 avg=2.81
[4980 | 16474.30] loss=2.80 avg=2.81
[4990 | 16506.15] loss=3.17 avg=2.81
[5000 | 16538.03] loss=3.23 avg=2.82
Saving ../models/deep-metal/checkpoints/run1/model-5000


## Post-training jobs

- Move the checkpoints inside the folder that will become a new model: `deep-metal`

In [8]:
source = f"{CHECKPOINT_DIR}/{RUN_NAME}/"
dest1 = f"{MODEL_DIR}/{FINETUNED_MODEL_NAME}"
files = os.listdir(source)

for f in files:
    shutil.copy(source + f, dest1)

In [4]:
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(
    sess, 
    run_name=RUN_NAME,
    checkpoint_dir=CHECKPOINT_DIR,
    model_name=FINETUNED_MODEL_NAME,
    model_dir=MODEL_DIR, 
)

Loading pretrained model ../models/deep-metal/model-5000
INFO:tensorflow:Restoring parameters from ../models/deep-metal/model-5000


## Generate lyrics

- Load the model and sample 10 texts unconditionally
- Sample 10 text using the incipit of a famous heavy metal song (_Symphony of Destruction_ by **Megadeth**)

In [5]:
def generate_lyrics(nsamples=1, prefix="<|startoftext|>", length=1023):
    if prefix == "<|startoftext|>":
        include_prefix = False
    else:
        include_prefix = True

    return gpt2.generate(
        sess,
        run_name=RUN_NAME,
        truncate='<|endoftext|>',
        destination_path=None,
        model_name=FINETUNED_MODEL_NAME,
        model_dir=MODEL_DIR,
        sample_dir=SAMPLE_DIR,
        sample_delim='=' * 20 + '\n',
        nsamples=nsamples,
        batch_size=2,
        length=length,
        temperature=0.8,
        prefix=prefix,
        include_prefix=include_prefix,
        return_as_list=True
    )

In [6]:
unconditionated_lyrics = generate_lyrics(nsamples=10, length=896)
for index, text in enumerate(unconditionated_lyrics, 1):
    print(f"========== SAMPLE {index} ==========")
    print("\n")
    print(text)
    print("\n\n")

KeyboardInterrupt: 

In [33]:
conditionated_lyrics = generate_lyrics(
    nsamples=6,
    prefix="You take a mortal man",
    length=1023
)
for index, text in enumerate(conditionated_lyrics, 1):
    print(f"========== SAMPLE {index} ==========")
    print("\n")
    print(text)
    print("\n\n")

========== SAMPLE 1 ==========


You take a mortal man
I've lost my way
You need blood, you need blood
You want me dead
You want me dead
I'm just a prisoner

I take a mortal man
I've lost my way
You need blood, you need blood
You want me dead
You want me dead
I'm just a prisoner

To kill my enemies
I feel my fall
Please your god
Please your god
You've got the right to choose
A path you choose
What's your price
Your life

You take a mortal man
I've lost my way
You need blood, you need blood
You want me dead
You want me dead
I'm just a prisoner

I take a mortal man
I've lost my way
You need blood, you need blood
You want me dead
You want me dead
I'm just a prisoner

To kill my enemies
I feel my fall
Please your god
Please your god
You've got the right to choose
A path you choose
What's your price
Your life

You make matters worse
You make matters worse
You make matters worse
You make matters worse
You make matters worse
You make matters worse
You make matters worse
You make matters worse